# Transformer Model

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# https://huggingface.co/docs/transformers/training

In [ ]:
df = pd.read_csv('untitled.csv')

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['IsBodyShopRepair'].value_counts()

In [ ]:
df.shape

In [ ]:
import random

In [ ]:
f = lambda x: (random.randint(50,100)/random.randint(5,10))

In [ ]:
f(1)

In [ ]:
df['IsBodyShopRepair'] = df['IsBodyShopRepair'].apply(f)

In [ ]:
import string
def clean(x):
    # x = x.tolist()[0]
    x = x.lower()
    new_str = ''
    for char in x:
        if char not in string.punctuation:
            new_str = new_str + char
    return new_str.strip()

In [ ]:
df['text'] = df['Damage'].apply(lambda x: clean(x))

In [ ]:
df.rename(columns={'IsBodyShopRepair':'label'},inplace=True)

In [ ]:
df.drop('Damage',axis=1,inplace=True)

In [ ]:
df['label'] = df['label'].replace(['Y','N'],[1,0])

In [ ]:
df

In [ ]:
# df['label'].unique()

In [ ]:
!pip install -q datasets

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train,df_val = train_test_split(df,test_size=0.1,random_state=42)

In [ ]:
df_train.shape

In [ ]:
df_val.shape

In [ ]:
import datasets

In [ ]:
import datasets
import os
# import numpy as np


# prepare dataset for finetuning
train_dataset = datasets.Dataset.from_dict(df_train)
val_dataset = datasets.Dataset.from_dict(df_val)


my_dataset_dict = datasets.DatasetDict({"train":train_dataset,"val":val_dataset})

In [ ]:
my_dataset_dict

In [ ]:
# !pip install -q transformers

In [ ]:
from transformers import AutoTokenizer

In [ ]:
# load Tokenizer from bert-base-uncased checkpoint
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
tokenizer

In [ ]:
# get tokenized dataset
def tokenize_function(examples):
    # max token length max_length=20, you can change it to max 512
    return tokenizer(examples["text"], padding="max_length", max_length=20, truncation=True)
tokenized_datasets = my_dataset_dict.map(tokenize_function, batched=True)

In [ ]:
my_dataset_dict

In [ ]:
tokenized_datasets

In [ ]:
df['label'].unique().tolist()

In [ ]:
class_names = df['label'].unique().tolist()

In [ ]:
from transformers import AutoModelForSequenceClassification

# load classifier from bert-base-uncased
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(class_names))

In [ ]:
model

In [ ]:
!pip install -q evaluate

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
# function to compute performance while finetuning
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer
# prepare training Arguments
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",
                                  num_train_epochs = 2, # change here
                                  overwrite_output_dir=True)

In [ ]:
pip install --upgrade accelerate

In [ ]:
# Trainer for finetuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    compute_metrics=compute_metrics,
    # callbacks=[checkpoint_callback]
)

In [ ]:
# train the model for finetuning
trainer.train()

In [ ]:
# save the model
trainer.save_model("MyOwn_finetuned_model")

# Now use Our finetuned Model

In [ ]:
from transformers import (
    AutoModel, AutoConfig, 
    AutoTokenizer, logging
)

# load fientuned model
model_tuned = AutoModelForSequenceClassification.from_pretrained("MyOwn_finetuned_model")

In [ ]:
model_tuned

In [ ]:
from scipy.special import softmax
def inference(text):
    batch = tokenizer.encode(text, return_tensors='pt')
    output = model_tuned(batch)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    labels = [0,1]
    return labels[scores.argmax()]

In [ ]:
# prediction for first 50 lines
df['text'][:50].apply(lambda x:inference(x))

# test

In [27]:
import pandas as pd
import random
from sklearn.preprocessing import MinMaxScaler

In [14]:
f = lambda : random.randint(50,70)/random.randint(2,10)

In [16]:
d = {'col1':[],'col2':[]}

for i in range(100):
    d['col1'].append(f())
    d['col2'].append(f())

In [19]:
df = pd.DataFrame(d)

In [32]:
df

,col1,col2
0,17.250000,9.833333
1,9.714286,8.666667
2,7.250000,31.000000
3,13.750000,6.600000
4,7.428571,14.250000
...,...,...
95,14.500000,6.600000
96,19.333333,11.800000
97,10.833333,20.000000
98,12.200000,11.333333


In [40]:
from sklearn.model_selection import train_test_split

In [41]:
X_train, X_test = train_test_split(df,test_size=0.33, random_state=42)

In [42]:
sc = MinMaxScaler()

In [43]:
X_train = sc.fit_transform(X_train)

In [44]:
X_train

array([[0.21070234, 0.45011338],
       [0.        , 0.91496599],
       [0.06772575, 0.15694849],
       [0.24414716, 0.91496599],
       [0.25083612, 0.03741497],
       [0.7993311 , 0.05328798],
       [0.14476828, 0.24829932],
       [0.66555184, 0.16666667],
       [0.0449647 , 0.96598639],
       [0.0367893 , 0.51814059],
       [0.31438127, 0.05102041],
       [0.30602007, 0.3877551 ],
       [0.03381643, 0.20748299],
       [0.0449647 , 0.4047619 ],
       [0.05239688, 0.50680272],
       [0.41471572, 0.03401361],
       [0.12207358, 0.07434402],
       [0.13043478, 0.60884354],
       [0.28929766, 0.05102041],
       [0.05982906, 0.76190476],
       [0.09280936, 0.0457294 ],
       [0.39799331, 0.10714286],
       [0.46488294, 0.25170068],
       [0.07608696, 0.06840514],
       [0.12486065, 0.04950869],
       [0.56521739, 0.04081633],
       [0.5206243 , 0.00340136],
       [0.13600892, 0.79591837],
       [0.27090301, 0.89795918],
       [0.43143813, 0.13265306],
       [0.

In [45]:
sc.transform(X_test)

array([[ 0.98327759,  0.06887755],
       [ 0.06354515,  0.10714286],
       [ 0.54292085,  0.23469388],
       [ 0.71571906,  0.0457294 ],
       [ 0.19175028,  0.48412698],
       [ 0.10654563,  0.77891156],
       [ 0.83277592,  0.02305367],
       [ 0.11371237,  0.12131519],
       [ 0.07840951,  0.08892128],
       [ 0.40635452,  0.16099773],
       [ 0.69899666,  0.29421769],
       [ 0.34782609,  0.3877551 ],
       [ 0.17725753,  0.91496599],
       [ 0.10535117,  0.3792517 ],
       [ 0.17502787,  0.0170068 ],
       [ 0.07787864,  0.31122449],
       [ 0.06832298,  0.00680272],
       [ 0.46488294,  0.00340136],
       [ 0.17502787,  0.06084656],
       [ 0.1638796 ,  0.01020408],
       [ 0.27257525,  0.29591837],
       [ 0.23076923,  0.89795918],
       [ 0.12486065,  0.06462585],
       [-0.00334448,  0.07974301],
       [ 0.14954611, -0.00340136],
       [ 0.7826087 ,  0.09438776],
       [ 0.05351171,  0.09378037],
       [ 0.47603122,  0.22789116],
       [ 0.56521739,

In [46]:
f = lambda : random.randint(50,700)/random.randint(2,10)

d = {'col1':[],'col2':[]}

for i in range(100):
    d['col1'].append(f())
    d['col2'].append(f())
    
df = pd.DataFrame(d)

In [48]:
sc.transform(df)

array([[ 0.13043478,  2.56705539],
       [ 1.99219621,  2.13945578],
       [ 6.4960981 ,  1.75963719],
       [ 1.90301003,  2.71768707],
       [ 2.54960981,  6.11904762],
       [ 4.25250836,  2.26795163],
       [ 4.07023411,  2.68367347],
       [ 1.21404682,  0.89067055],
       [ 0.70642884,  1.22534014],
       [ 1.70234114,  5.76190476],
       [ 0.44816054,  3.11451247],
       [ 0.83277592,  1.0085034 ],
       [ 8.60869565,  7.17346939],
       [ 6.11705686,  2.36479592],
       [ 1.34559643,  4.41836735],
       [ 1.70711897,  2.86054422],
       [ 3.16276477,  9.63945578],
       [ 1.03716091,  1.51020408],
       [ 0.84392419,  5.1553288 ],
       [ 6.19509476,  1.67298348],
       [ 1.05852843,  2.71282799],
       [ 4.16889632,  0.4047619 ],
       [ 0.87179487,  3.15419501],
       [ 7.43255295,  4.41836735],
       [ 1.3812709 ,  3.64739229],
       [ 1.48256092,  1.87490552],
       [ 6.65217391,  4.19160998],
       [ 9.71237458,  0.86961451],
       [ 2.09698997,